# Example to Read / Write to Cassandra with Spark

Documentation: https://github.com/datastax/spark-cassandra-connector/blob/master/doc/15_python.md


In [1]:
import pyspark
from pyspark.sql import SparkSession

In [34]:
# CASSANDRA CONFIGURATION
cassandra_host = "cassandra"

In [4]:
# Spark init
spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
      .config("spark.cassandra.connection.host", cassandra_host) \
      .config("spark.jars.packages","com.datastax.spark:spark-cassandra-connector-assembly_2.12:3.1.0")\
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.datastax.spark#spark-cassandra-connector-assembly_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-653d0eb2-bc9f-45e3-84d9-25a0b47973b6;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.spark#spark-cassandra-connector-assembly_2.12;3.1.0 in central
downloading https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector-assembly_2.12/3.1.0/spark-cassandra-connector-assembly_2.12-3.1.0.jar ...
	[SUCCESSFUL ] com.datastax.spark#spark-cassandra-connector-assembly_2.12;3.1.0!spark-cassandra-connector-assembly_2.12.jar (396ms)
:: resolution report :: resolve 462ms :: artifacts dl 398ms
	:: modules in use:
	com.datastax.spark#spark-cassandra-connector-assembly_2.12;3.1.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   1   |   1   |   0   ||   1   |   1   |
	---------------------------------------------------------------------
:: retrieving :: org.apac

In [5]:
# read local data
df = spark.read.option("multiline","true").json("/home/jovyan/datasets/json-samples/stocks.json")
df.toPandas()

,price,symbol
0,126.82,AAPL
1,3098.12,AMZN
2,251.11,FB
3,1725.05,GOOG
4,128.39,IBM
5,212.55,MSFT
6,78.00,NET
7,497.00,NFLX
8,823.80,TSLA
9,45.11,TWTR


In [31]:
# WE NEED A TABLE BEFORE WE CAN WRITE, Using Plain old Python
!pip install -q cassandra-driver
from cassandra.cluster import Cluster
with Cluster([cassandra_host]) as cluster:
    session = cluster.connect()
    session.execute("CREATE KEYSPACE IF NOT EXISTS example WITH replication={ 'class': 'SimpleStrategy', 'replication_factor' : 1 };")
    session.execute("CREATE TABLE IF NOT EXISTS example.stocks (symbol text, price decimal, primary key (symbol));")

In [32]:
# Write to back to our newly-minted Cassandra table, Append mode is okay here because of Cassandra's default upsert behavior.
df.write.format("org.apache.spark.sql.cassandra")\
  .mode("Append")\
  .option("table", "stocks")\
  .option("keyspace","example")\
  .save()

In [33]:
# read back from Cassandra
df1 =spark.read.format("org.apache.spark.sql.cassandra")\
    .options(table="stocks", keyspace="example") \
    .load()
df1.toPandas()

,symbol,price
0,TWTR,45.110000000000000000
1,IBM,128.390000000000000000
2,FB,251.110000000000000000
3,NFLX,497.000000000000000000
4,NET,78.000000000000000000
5,AAPL,126.820000000000000000
6,AMZN,3098.120000000000000000
7,TSLA,823.800000000000000000
8,GOOG,1725.050000000000000000
9,MSFT,212.550000000000000000
